In [2]:
import pandas as pd

In [ ]:
data=pd.read_csv("data/reddit2008/comments_2008_asm_v1.1/comments_2008/csv-0.csv").append(pd.read_csv("data/reddit2008/comments_2008_asm_v1.1/comments_2008/csv-1.csv")).append(pd.read_csv("data/reddit2008/comments_2008_asm_v1.1/comments_2008/csv-2.csv")).append(pd.read_csv("data/reddit2008/comments_2008_asm_v1.1/comments_2008/csv-3.csv")).append(pd.read_csv("data/reddit2008/comments_2008_asm_v1.1/comments_2008/csv-4.csv")).append(pd.read_csv("data/reddit2008/comments_2008_asm_v1.1/comments_2008/csv-5.csv")).append(pd.read_csv("data/reddit2008/comments_2008_asm_v1.1/comments_2008/csv-6.csv")).append(pd.read_csv("data/reddit2008/comments_2008_asm_v1.1/comments_2008/csv-7.csv")).append(pd.read_csv("data/reddit2008/comments_2008_asm_v1.1/comments_2008/csv-8.csv")).append(pd.read_csv("data/reddit2008/comments_2008_asm_v1.1/comments_2008/csv-9.csv")).append(pd.read_csv("data/reddit2008/comments_2008_asm_v1.1/comments_2008/csv-10.csv")).append(pd.read_csv("data/reddit2008/comments_2008_asm_v1.1/comments_2008/csv-11.csv"))

data.reset_index(drop=True, inplace=True)

data = data.iloc[:,1:] #Zanemari Unnamed kolonu
print(data)

In [4]:
ima_autora=data['author']=='[deleted]'
print(ima_autora)
data_noAuth=data[ima_autora]
labele=['id','author','link_id']
data_bezAutora=data_noAuth[labele]
print(data_bezAutora)

0          False
1           True
2          False
3           True
4           True
           ...  
7242866     True
7242867    False
7242868     True
7242869    False
7242870    False
Name: author, Length: 7242871, dtype: bool
              id     author   link_id
1        c02s9s7  [deleted]  t3_647ht
3        c02s9s9  [deleted]  t3_648oo
4        c02s9sa  [deleted]  t3_648et
5        c02s9sb  [deleted]  t3_648iy
15       c02s9sl  [deleted]  t3_648et
...          ...        ...       ...
7242857  c06vwu6  [deleted]  t3_7k76h
7242860  c06vwu9  [deleted]  t3_7k6lh
7242862  c06vwub  [deleted]  t3_7k8en
7242866  c06vwuf  [deleted]  t3_7k8lt
7242868  c06vwuh  [deleted]  t3_7k31f

[2369181 rows x 3 columns]


In [ ]:
print(data['created_utc'])

In [ ]:
print(data_bezAutora.index.shape[0])
print(data_bezAutora['id'])

In [ ]:
data.drop(data[data['author']=='[deleted]'].index, inplace=True)
print(data)

In [8]:
print(data['id'].isnull().values.any())   #Id kolona ima prazne kolone
print(data['id'].is_unique)     #Id kolona ima sve jedinstvene vrednosti

True
True


In [9]:
dataNull=data['id'].isnull()
print(data[dataNull])

          id     author   link_id parent_id  created_utc   subreddit  \
6422487  NaN  fuzzybunn  t3_7kef0    t1_k7p   1229661971  reddit.com   

        subreddit_id  score distinguished  gilded  controversiality  
6422487         t5_6      1           NaN       0                 0  


In [ ]:
data.drop(data[dataNull].index, inplace=True)
print(data)

In [11]:
data.isna().any()          #sve je u redu, osim za distinguished polje koje moze biti null

id                  False
author              False
link_id             False
parent_id           False
created_utc         False
subreddit           False
subreddit_id        False
score               False
distinguished        True
gilded              False
controversiality    False
dtype: bool

In [12]:
print(data.dtypes)   #distinguished polje sumnjivo

id                  object
author              object
link_id             object
parent_id           object
created_utc          int64
subreddit           object
subreddit_id        object
score                int64
distinguished       object
gilded               int64
controversiality     int64
dtype: object


In [13]:
print(data.index.is_unique) #u redu

True


In [14]:
print(data['id'].isnull().values.any())   #Id kolona nema prazne kolone, u redu

False


In [15]:
print(data.shape)

(4873689, 11)


## Čišćenje komentara izbrisanih objava

In [16]:
import pickle

In [17]:
data_comments = data

In [18]:
with open("data/submissions_cleaned", 'rb') as file:
    data_submissions=pickle.load(file)

In [ ]:
print(data_submissions)

In [ ]:
print(data_comments)

In [ ]:
idObj=data_submissions[['id']]
print(idObj)

In [22]:
t3Id='t3_'+ idObj

In [ ]:
print(t3Id)

In [24]:
t3Id.rename(columns = {'id':'link_id'}, inplace = True)

In [ ]:
print(t3Id)

In [26]:
data_comments1=data_comments.merge(t3Id, how='inner', on='link_id')

In [ ]:
data_comments1=data_comments1.sort_index()
print(data_comments1)    #uklonjeni komentari izbrisanih objava

In [28]:
print(data_comments1.shape)

(3782277, 11)


## Čišćenje izbrisanih komentara

In [ ]:
#kolona id-eva kojoj je dodat prefix 't1_' da bi se uradio presek sa kolonom 'parent_id'
idComm='t1_'+data_comments1[['id']]

idComm.rename(columns = {'id':'parent_id'}, inplace = True)

print(idComm)
print(type(idComm))

In [ ]:
#presek
#ocisceni komentari - nakon brisanja komentara ciji je parent_id komentar na obrisanu objavu
data_commentsKomentari=data_comments1.merge(idComm, how='inner', on='parent_id')
#komentari nad validnim objavama
data_commentsObjave = data_comments1[data_comments1["parent_id"].str.contains('t3_')==True]

#unija
data_commentsOut = pd.concat([data_commentsKomentari, data_commentsObjave])
data_commentsOut=data_commentsOut.sort_values(by='created_utc', ascending=True)
data_commentsOut.reset_index(drop=True, inplace=True)
data_commentsOut=data_commentsOut.sort_index()
print(data_commentsOut)

In [39]:
print(data_commentsOut.shape)

(3209446, 11)


In [41]:
with open("data/comments_cleaned", 'wb') as file:
    pickle.dump(data_commentsOut, file)